# Create the features DF

# Import

In [42]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 

from scipy import sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [23]:
df = pd.read_pickle ('./data/FL_only.pkl')
LDA_df = pd.read_pickle ('./data/LDA_training.pkl')

In [24]:
df.shape

(23630, 18)

In [25]:
LDA_df.shape

(323563, 18)

In [26]:
LDA_df.head()

,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,stars_y,review_count,is_open,attributes,categories,hours
2280,cg4JFJcCxRTTMmcg9O9KtA,1.0,0,1,0,Skip this train wreck if you are looking for d...,2018-02-11 03:11:20,Ava,718 S Howard Ave,Tampa,FL,33606,3.5,544,1,"{'BikeParking': 'True', 'Alcohol': ''full_bar'...","Wine Bars, Desserts, Bars, American (New), Nig...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
2281,cg4JFJcCxRTTMmcg9O9KtA,4.0,0,0,0,Been here twice and both times we thoroughly e...,2018-07-26 14:30:20,Ava,718 S Howard Ave,Tampa,FL,33606,3.5,544,1,"{'BikeParking': 'True', 'Alcohol': ''full_bar'...","Wine Bars, Desserts, Bars, American (New), Nig...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
2282,cg4JFJcCxRTTMmcg9O9KtA,5.0,0,0,0,Brunch was lovely at Ava. No wait and fast ser...,2018-08-26 20:32:05,Ava,718 S Howard Ave,Tampa,FL,33606,3.5,544,1,"{'BikeParking': 'True', 'Alcohol': ''full_bar'...","Wine Bars, Desserts, Bars, American (New), Nig...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
2283,cg4JFJcCxRTTMmcg9O9KtA,3.0,0,0,1,Just walk up and order then go around and wait...,2018-03-24 20:10:35,Ava,718 S Howard Ave,Tampa,FL,33606,3.5,544,1,"{'BikeParking': 'True', 'Alcohol': ''full_bar'...","Wine Bars, Desserts, Bars, American (New), Nig...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."
2284,cg4JFJcCxRTTMmcg9O9KtA,1.0,0,0,0,First time coming and the last time this place...,2018-09-16 00:00:28,Ava,718 S Howard Ave,Tampa,FL,33606,3.5,544,1,"{'BikeParking': 'True', 'Alcohol': ''full_bar'...","Wine Bars, Desserts, Bars, American (New), Nig...","{'Monday': '17:0-22:0', 'Tuesday': '17:0-22:0'..."


# Preprocessing 

In [27]:
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

In [28]:
stop_words = stopwords.words('english')

def preprocess(text : str) -> list:
    
    tokens = text.lower().rstrip(' ').rstrip('\n').rstrip('.').split(' ')[:3000]
    
    p = PorterStemmer()
    stemmed_doc =[]
    for token in tokens:
        stemmed_doc.append(p.stem(token))
        
    tokens = [i.split(' ') for i in stemmed_doc]
    
    
    final_tokens =[]


    for l in tokens:
        for word in l:
            word = word.rstrip('.').rstrip('\'').rstrip(',')
            if word not in final_tokens and word not in stop_words and len(word)>0:
                final_tokens.append(word)
    return final_tokens

# Review-based Market Attractiveness Features (MAF)
* Turn every restaurant and city to a doc of reviews
* Use `dict` to store the dic index of restaurant and city
* review - LDA
* location - LDA 

In [29]:
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import nltk

In [30]:
# # aspect creation 
# total_df = pd.concat([df,LDA_df] , ignore_index=True)

# docs = []
# index =0 


# # restaurant
# for restaurant in Counter(df.name):
#     if index%10 ==0:
#         print(f'Now {index} Restaurant ... {restaurant}')
#     tmp = df[df.name == restaurant]
#     texts = ''.join(tmp.text.tolist())
#     docs.append(preprocess(texts))
#     indxe +=1

# # city 
# for city_name in Counter(df.city):
#     if index%5 ==0:
#         print(f'Now {index} City ... {city_name}')
#     tmp = total_df[total_df.city == city_name]
#     texts = ''.join(tmp.text.tolist())
#     docs.append(preprocess(texts))
#     index +=1   

# # save pickle
# with open('./data/docs.pkl', 'wb') as f:
#     pickle.dump(docs, f)

In [31]:
with open('./data/docs.pkl', 'rb') as f:
    docs = pickle.load(f)
    

## LDA

In [36]:
index = 0
name_city_dict = {}
for restaurant in Counter(df.name):
    name_city_dict[restaurant]=index
    index +=1
    
for city_name in Counter(df.city):
    name_city_dict[city_name]=index
    index +=1   

In [32]:
### LDA ###
common_dictionary = corpora.Dictionary(docs)
common_corpus = [common_dictionary.doc2bow(text) for text in docs]


# Train the model on the corpus.
lda =  gensim.models.LdaMulticore(corpus = common_corpus, id2word= common_dictionary, num_topics= 10)
lda.show_topics()


[(0,
  '0.001*"love" + 0.001*"experi" + 0.001*"order" + 0.001*"make" + 0.001*"visit" + 0.001*"small" + 0.001*"need" + 0.001*"i\'v" + 0.001*"want" + 0.001*"serv"'),
 (1,
  '0.002*"servic" + 0.001*"food" + 0.001*"nice" + 0.001*"peopl" + 0.001*"tabl" + 0.001*"5" + 0.001*"someth" + 0.001*"order" + 0.001*"alway" + 0.001*"wa"'),
 (2,
  '0.001*"could" + 0.001*"servic" + 0.001*"good" + 0.001*"best" + 0.001*"also" + 0.001*"told" + 0.001*"cold" + 0.001*"go" + 0.001*"everi" + 0.001*"around"'),
 (3,
  '0.002*"veri" + 0.001*"best" + 0.001*"great" + 0.001*"stop" + 0.001*"i\'m" + 0.001*"check" + 0.001*"someth" + 0.001*"realli" + 0.001*"befor" + 0.001*"get"'),
 (4,
  '0.001*"also" + 0.001*"staff" + 0.001*"thi" + 0.001*"ask" + 0.001*"come" + 0.001*"wa" + 0.001*"would" + 0.001*"food" + 0.001*"never" + 0.001*"2"'),
 (5,
  '0.002*"back" + 0.001*"one" + 0.001*"look" + 0.001*"well" + 0.001*"onli" + 0.001*"ever" + 0.001*"2" + 0.001*"manag" + 0.001*"never" + 0.001*"come"'),
 (6,
  '0.001*"go" + 0.001*"got" + 

In [33]:
# check the topic coherence
coherence_model_lda = CoherenceModel( model=lda , texts=docs , dictionary=common_dictionary , coherence='c_v' )
conherence_lda = coherence_model_lda.get_coherence()
print(conherence_lda)

0.5555102731375688


In [37]:
# create restaurant aspects

rest_aspect = {}
loca_aspect = {}

for name in Counter(df.name):
    tmp_arr = np.zeros(10)
    for i in lda[common_corpus[name_city_dict[name]]]:
        tmp_arr[i[0]]=i[1]
    rest_aspect[name]= tmp_arr

for location in Counter(df.city):
    tmp_arr = np.zeros(10)
    for i in lda[common_corpus[name_city_dict[location]]]:
        tmp_arr[i[0]]=i[1]
    loca_aspect[location]= tmp_arr

## Affinity & Complementary

In [38]:
df = df.reset_index()

In [39]:
df['affinity']=""
df['complementary']=""

for i in range(len(df)):
    df.affinity[i] = rest_aspect[df.name[i]]*loca_aspect[df.city[i]]
    df.complementary[i] = rest_aspect[df.name[i]]*(1-loca_aspect[df.city[i]]) 

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
df.head()

,index,business_id,stars_x,useful,funny,cool,text,date,name,address,...,state,postal_code,stars_y,review_count,is_open,attributes,categories,hours,affinity,complementary
0,49609,j9Kaj_6tSeXmVMYCgZithg,5.0,0,0,0,Great food! Very impressed...I really like the...,2018-05-19 02:19:28,Noble Crust,28330 Paseo Dr,...,FL,33543,4.5,496,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Restaurants, Italian, Nightlife, Southern, Bar...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...","[0.0, 0.0, 0.0, 0.0, 0.8171585500305412, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.08329090475385215, 0.0,..."
1,49610,j9Kaj_6tSeXmVMYCgZithg,2.0,0,0,0,Not a fan. Italian food is pretty straight for...,2018-09-14 13:10:27,Noble Crust,28330 Paseo Dr,...,FL,33543,4.5,496,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Restaurants, Italian, Nightlife, Southern, Bar...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...","[0.0, 0.0, 0.0, 0.0, 0.8171585500305412, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.08329090475385215, 0.0,..."
2,49611,j9Kaj_6tSeXmVMYCgZithg,5.0,0,0,0,Noble crust is amazing! The atmosphere is quit...,2018-01-12 01:40:44,Noble Crust,28330 Paseo Dr,...,FL,33543,4.5,496,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Restaurants, Italian, Nightlife, Southern, Bar...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...","[0.0, 0.0, 0.0, 0.0, 0.8171585500305412, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.08329090475385215, 0.0,..."
3,49612,j9Kaj_6tSeXmVMYCgZithg,3.0,0,0,0,"Meh, expensive and uncomfortable seating. One ...",2018-08-05 04:03:04,Noble Crust,28330 Paseo Dr,...,FL,33543,4.5,496,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Restaurants, Italian, Nightlife, Southern, Bar...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...","[0.0, 0.0, 0.0, 0.0, 0.8171585500305412, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.08329090475385215, 0.0,..."
4,49613,j9Kaj_6tSeXmVMYCgZithg,5.0,0,0,0,Great date night place! I can't say enough abo...,2018-02-03 04:05:37,Noble Crust,28330 Paseo Dr,...,FL,33543,4.5,496,1,"{'BusinessAcceptsCreditCards': 'True', 'DogsAl...","Restaurants, Italian, Nightlife, Southern, Bar...","{'Monday': '16:0-22:0', 'Tuesday': '16:0-22:0'...","[0.0, 0.0, 0.0, 0.0, 0.8171585500305412, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.08329090475385215, 0.0,..."


# Review-based Market Competitiveness Feature (MCF)
* review - LDA rank 

# Geographic Features (GeoF)
* Density : the restaurant number in the same postal code
* Neighbourhood Entropy : entropy measure of the frequency of restaurant categories
    * A location with **higher entropy value** is expected to be **more diverse** in terms of restaurant categories,
* Competitiveness : the proportion of nearby restaurants with the same type of restaurant 
* Jensen Quality : Jensen Quality of each venue category as a feature value, rather than simply aggregating Jensen Quality values over all venue categories as a single feature value. 

In [41]:
# calculate the density by Postal code
df['density']=""
for i in range(len(df)):
    tmp = df[df.city==df.city[i]]
    df.density[i] = len(Counter(tmp.name))

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# Neighbourhood Entropy 
df['entropy']=""

for i in range(len(df)):
    